<a href="https://colab.research.google.com/github/kridtapon/Klinger-ATR-Fusion/blob/main/Klinger_ATR_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00


In [2]:
pip install --upgrade yfinance

In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Klinger Oscillator
def calculate_klinger(df, short_period=34, long_period=55):
    """
    Calculate the Klinger Oscillator.
    """
    # Short-term and Long-term moving averages of volume
    df['Short_MA'] = df['Volume'].ewm(span=short_period).mean()
    df['Long_MA'] = df['Volume'].ewm(span=long_period).mean()

    # Klinger Oscillator (Price * Volume Difference)
    df['Klinger'] = (df['Volume'] * (df['Close'] - df['Close'].shift(1))).cumsum() - \
                    (df['Short_MA'] - df['Long_MA']).cumsum()
    return df

# Function to calculate Average True Range (ATR)
def calculate_atr(df, period=14):
    """
    Calculate Average True Range (ATR).
    """
    df['H-L'] = df['High'] - df['Low']
    df['H-C'] = abs(df['High'] - df['Close'].shift(1))
    df['L-C'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-C', 'L-C']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period).mean()
    return df

# Define the stock symbol and time period
symbol = 'META'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Calculate Klinger Oscillator
df = calculate_klinger(df)

# Calculate Average True Range (ATR)
df = calculate_atr(df)

# Define Entry and Exit signals based on Klinger Oscillator and ATR
df['Entry'] = (df['Klinger'] > 0) & (df['ATR'] > df['ATR'].shift(1))  # Klinger Oscillator > 0 and ATR increasing
df['Exit'] = (df['Klinger'] < 0) & (df['ATR'] < df['ATR'].shift(1))   # Klinger Oscillator < 0 and ATR decreasing

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                            90649.241127
Total Return [%]                        -9.350759
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                         530.79548
Max Drawdown [%]                        45.752489
Max Drawdown Duration                       834.0
Total Trades                                    3
Total Closed Trades                             3
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            33.333333
Best Trade [%]                          41.434323
Worst Trade [%]                        -29.572828
Avg Winning Trade [%]                   41.434323
Avg Losing Trade [%]                   -19.293778
